In [ ]:
!pip install torch torchvision rasterio geopandas
!pip install -U "datasets<4.0.0" huggingface_hub==0.27.0


In [ ]:
from datasets import load_dataset

In [ ]:
# use streaming because big dataset
ds = load_dataset("prs-eth/AGBD", trust_remote_code=True, streaming=True)
train_ds = ds["train"]
val_ds   = ds["validation"]

# **attention u-net**
its like u-net but with attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#this attention part of attention u-net
#who could have thought
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(in_channels = in_dim, out_channels = in_dim//8, kernel_size = 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim, out_channels = in_dim//8, kernel_size = 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim, out_channels = in_dim, kernel_size = 1)

        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        m_batchsize, C, width, height = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = torch.softmax(energy, dim = -1)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, width, height)

        out = self.gamma*out + x
        return out, attention

#its conv
#but twice
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)
#conv down
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

#conv up
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

#originally had 4 up/down layers but that would downsample it to 0 so it has 3 now instead
class Attention_UNet(nn.Module):
    def __init__(self, n_channels, n_classes, middle_layers=[64, 128, 256, 512], bilinear=False):
        if len(middle_layers) != 4:  # Changed from 5 to 4
            raise ValueError("middle_layers must be a list of length 4")

        super(Attention_UNet, self).__init__()
        self.n_channels = n_channels
        self.middle_layers = middle_layers
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, middle_layers[0]))  # Changed from n_classes to n_channels
        self.down1 = (Down(middle_layers[0], middle_layers[1]))
        self.down2 = (Down(middle_layers[1], middle_layers[2]))
        self.attention_1 = SelfAttention(middle_layers[0])
        self.attention_2 = SelfAttention(middle_layers[1])
        self.attention_3 = SelfAttention(middle_layers[2])

        factor = 2 if bilinear else 1
        self.down3 = (Down(middle_layers[2], middle_layers[3] // factor))  # Changed from down4 to down3
        self.up1 = (Up(middle_layers[3], middle_layers[2] // factor, bilinear))  # Changed from up1
        self.up2 = (Up(middle_layers[2], middle_layers[1] // factor, bilinear))  # Changed from up2
        self.up3 = (Up(middle_layers[1], middle_layers[0], bilinear))  # Changed from up3 (previously up4)
        self.outc = (OutConv(middle_layers[0], n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)  # Changed from x5 = self.down4(x4)
        v1, _ = self.attention_3(x3)  # Changed from attention_4(x4)
        v2, _ = self.attention_2(x2)
        v3, _ = self.attention_1(x1)
        x = self.up1(x4, v1)  # Changed from x5 to x4
        x = self.up2(x, v2)
        x = self.up3(x, v3)  # Removed the up4 step
        logits = self.outc(x)
        return logits

# **training**

In [ ]:
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader

In [ ]:
import numpy as np

sample = next(iter(train_ds))
print(sample.keys())

features = np.array(sample["input"])
print("shape:", features.shape)

dict_keys(['input', 'label'])
shape: (24, 15, 15)


In [ ]:
def collate_fn(batch):
    X = torch.stack([torch.tensor(f['input']) for f in batch])
    y = torch.tensor([f["label"] for f in batch])
    return X.float(), y.float()

train_loader = DataLoader(train_ds, batch_size=32, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, collate_fn=collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Attention_UNet(n_channels=features.shape[0], n_classes=1).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_fn = nn.MSELoss()
scaler = GradScaler('cuda')

best_val_loss = float('inf')

In [ ]:
num_epochs = 3
for epoch in range(1, num_epochs+1):
    model.train()
    train_loss = 0.0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        with autocast('cuda'):
            out = model(X).squeeze(1)

            pred = out[:, out.shape[1]//2, out.shape[2]//2]
            loss = loss_fn(pred, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * X.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad(), autocast():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            out = model(X).squeeze(1)
            pred = out[:, out.shape[1]//2, out.shape[2]//2]
            loss = loss_fn(pred, y)
            val_loss += loss.item() * X.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch}/{num_epochs} — Train RMSE: {train_loss**0.5:.3f}, Val RMSE: {val_loss**0.5:.3f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_attunet_agb.pth")